# 동적 웹크롤링

### 라이브러리 설치

In [ ]:
!pip install selenium
!pip install bs4

## WebDriver 임포트

In [134]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [135]:
# 객체생성
wd = webdriver.Chrome()   # 자동화 브라우저 생성

In [136]:
# 자동화 브라우져에 주소를 입력
wd.get("https://www.coffeebeankorea.com/store/store.asp")

In [138]:
# 해당 주소에 있는 자바스크립트 실행
wd.execute_script("storePop2(3)")

In [ ]:
# html 소스로 변환
html = wd.page_source

In [ ]:
# 파싱을 위해 beautifulsoup 객체 생성
soupCB1 = BeautifulSoup(html,'html.parser')

In [ ]:
# html 확인 - 필요하면
# print(soupCB1.prettify())

In [ ]:
# 추출하려는 패턴을 확인하고 적용  매장명
store_title = soupCB1.select('div.store_txt>h2')
len(store_title) > 0

In [ ]:
# 매장정보
tr_lists = soupCB1.select('table.store_table>tbody>tr')

In [ ]:
print(tr_lists[0].th.text,  tr_lists[0].td.text)
print(tr_lists[1].th.text,  tr_lists[1].td.text)
print(tr_lists[2].th.text,  tr_lists[2].td.text)
print(tr_lists[3].th.text,  tr_lists[3].td.text)
print(tr_lists[4].th.text,  tr_lists[4].td.text)

In [140]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [142]:
# 동적페이지 크롤링
wd = webdriver.Chrome() 
wd.get("https://www.coffeebeankorea.com/store/store.asp")
time.sleep(1)  # 대기
for store_num in range(2):
    wd.execute_script(f"storePop2({store_num})")
    time.sleep(1)
    html = wd.page_source
    soupCB1 = BeautifulSoup(html,'html.parser')
    store_title = soupCB1.select('div.store_txt>h2')    
    print(store_title)
    # 버튼요소를 찾기
    button = WebDriverWait(wd,10).until(
        EC.presence_of_all_elements_located( (By.ID,'closeBtn1ActiveBtn'))
    )[0]
    button.click()

[]


TimeoutException: Message: 


In [146]:
# 버튼요소를 찾기
button = WebDriverWait(wd,1).until(
    EC.presence_of_all_elements_located( (By.ID,'closeBtn1ActiveBtn'))
)[0]
button.click()